## Transfer Learning

In [1]:
### CONVERT IMAGES TO.PNG

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
import math

%matplotlib inline
from __future__ import print_function
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, load_img

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import NASNetMobile
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet

import os
import cv2
import datetime


Using TensorFlow backend.


In [3]:
def get_model(name):
    if name == 'resnet50':
        model_choice = ResNet50(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))
        model_name = 'ResNet50'
        target_size = (224, 224)
        sfvector = (7, 7, 2048) ##shape_feature_vector
        reshape_feature_vector = (7* 7* 2048)
        optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=False)
        optim = "SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=False)"
        
    if name == 'vgg16':   
        model_choice = VGG16(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))
        model_name='VGG16'
        target_size = (224, 224)
        sfvector = (7, 7, 512) ##shape_feature_vector
        reshape_feature_vector = (7 * 7 * 512) 
        optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=5e-4, nesterov=False)
        optim = "SGD(lr=0.01, momentum=0.9, decay=5e-4, nesterov=False)"
        
    if name == 'vgg19':
        model_choice = VGG19(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))
        model_name='VGG19'
        target_size = (224, 224)
        sfvector = (7, 7, 512) ##shape_feature_vector
        reshape_feature_vector = (7 * 7 * 512) 
        optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=5e-4, nesterov=False)
        optim = "SGD(lr=0.01, momentum=0.9, decay=5e-4, nesterov=False)"
        
    if name == 'inceptionV3':
        model_choice = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
        model_name = 'InceptionV3'
        target_size = (299, 299)
        sfvector = (8, 8, 2048) ##shape_feature_vector
        reshape_feature_vector = (8* 8* 2048) 
        optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=4e-5, nesterov=False)
        optim = "SGD(lr=0.01, momentum=0.9, decay=4e-5, nesterov=False)"
    
    if name == 'xception':
        model_choice = Xception(weights= 'imagenet', include_top=False, input_shape=(299, 299, 3))
        model_name = 'Xception'
        target_size = (299, 299)
        sfvector = (10, 10, 2048) ##shape_feature_vector
        reshape_feature_vector = (10* 10* 2048)
        optimizer = keras.optimizers.SGD(lr=0.045, momentum=0.9, decay=1e-5, nesterov=False)
        optim = "SGD(lr=0.045, momentum=0.9, decay=1e-5, nesterov=False)"
     
    if name == 'densenet121':
        model_choice= DenseNet121(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))
        model_name = 'DenseNet121'
        target_size = (224, 224)
        sfvector = (7, 7, 1024)
        reshape_feature_vector = (7 * 7 * 1024)  
        optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
        optim = "SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)"
                                   
    if name == 'densenet169':
        model_choice= DenseNet169(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))
        model_name = 'DenseNet169'
        target_size = (224, 224)
        sfvector = (7, 7, 1664)
        reshape_feature_vector = (7 * 7 * 1664)
        optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
        optim = "SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)"
    
    if name == 'densenet201':
        model_choice = DenseNet201(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))
        model_name = 'DenseNet201'
        target_size = (224, 224)
        sfvector = (7, 7, 1920)
        reshape_feature_vector = (7 * 7 * 1920)
        optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
        optim = "SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)"
   
    return model_choice, model_name, reshape_feature_vector, sfvector, target_size, optimizer, optim

In [4]:
# CHOOSE THE MODEL YOU WANT TO USE
## get_model(name); 
## name = ('resnet50', 'vgg16', 'vgg19', 'inceptionV3', 'xception', 'densenet121', 'densenet169', 'densenet201')
model_choice, model_name, reshape_feature_vector, sfvector, target_size, optimizer, optim = get_model('vgg16')

In [5]:
# NUMBER OF EPOCHS, BATCH_SIZE
model_choice.summary()
epochs = 100
batch_size = 16

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
# WHERE IS YOUR WORKING DIR? CHANGE TO YOUR SPECS...
''' jupyter nbk or floyd hub? mode = ... '''

#mode = 'jupyter'
#mode = 'floyd'
mode = 'ipb'

if mode == 'floyd':
    BASE_DIR = '/img/'       
if mode == 'jupyter':
    BASE_DIR = 'C:/Users/Ana Brites/dataleuko3/' 
if mode == 'ipb':
    BASE_DIR = '/home/deep/Documents/Romeu_Beato/'

In [7]:
# WORK WITH FULL SIZE DATA OR TWEEK WITH SMALL ONE...
''' dataset size: dsize = ... '''

#dsize = 'small'
dsize = 'big'

if dsize == 'small':
    train_dir = BASE_DIR + 'images/TRAINsmall/'
    validation_dir = BASE_DIR +'images/TEST_SIMPLE2small/'
    test_dir = BASE_DIR + 'images/TEST_SIMPLEsmall/'
    nTrain = 395
    nVal = 40
    nTest = 40 
elif dsize == 'big': 
    validation_dir = BASE_DIR +'images/TEST_SIMPLE2/'
    train_dir = BASE_DIR + 'images/TRAIN/'
    test_dir = BASE_DIR + 'images/TEST_SIMPLE/'
    nTrain = 4394 ###
    nVal = 440 ###
    nTest = 440  ###

### Generate and save data

In [8]:
## BUILD GENERATOR AND CLASS INDICES

def build(source=None):
        datagen = ImageDataGenerator(rescale=1. / 255)
            #featurewise_center=True,
            #featurewise_std_normalization=True)
        data_generator = datagen.flow_from_directory(
        source,  # this is the target directory
        target_size=(target_size[0], target_size[1]),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
        class_dictionary = data_generator.class_indices
        return data_generator, class_dictionary

In [9]:
nTrain

4394

In [10]:
nTest

440

In [11]:
nVal

440

In [12]:
## Train generator, class dictionary, and train_features and train_labels

train_generator = [] 

train_features = np.zeros(shape=(nTrain, sfvector[0], sfvector[1], sfvector[2]))
train_labels = np.zeros(shape=(nTrain, 4))

train_generator = build(source=train_dir) 

i = 0
for inputs_batch, labels_batch in train_generator[0]:
    features_batch = model_choice.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain :
        break

train_features = np.reshape(train_features, (nTrain, reshape_feature_vector))

Found 4394 images belonging to 4 classes.


In [13]:
print(train_generator[1])

{'EOSINOPHIL': 0, 'LYMPHOCYTE': 1, 'MONOCYTE': 2, 'NEUTROPHIL': 3}


In [14]:
## validation generator, class dictionary, and validation_features and validation_labels

validation_generator = [] 
 
validation_features = np.zeros(shape=(nVal, sfvector[0], sfvector[1], sfvector[2]))
validation_labels = np.zeros(shape=(nVal,4))

validation_generator = build(source=validation_dir) 

i = 0
for inputs_batch, labels_batch in validation_generator[0]:
    features_batch = model_choice.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1 
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, sfvector[0] * sfvector[1] * sfvector[2]))

Found 440 images belonging to 4 classes.


#### Save Features

In [15]:
# save the output as a Numpy array. CHANGE IF NEEDED

#save_features = True
save_features= False

if save_features:
    np.save(open('bottleneck_features_train_{}.npy'.format(model_name), 'wb'), train_features) ### 'wb' or 'w'
    np.save(open('bottleneck_features_validation_{}.npy'.format(model_name), 'wb'), validation_features)

#### Load data

In [16]:
#LOAD DATA OR NOT?

#load_features = True
load_features = False

if load_features:
    train_features = np.load(open('bottleneck_features_train_{}.npy'.format(model_name),'rb'))
    validation_features = np.load(open('bottleneck_features_validation_{}.npy'.format(model_name),'rb'))

In [17]:
#print(train_features)
#print(train_labels)
#print(validation_features)
#print(validation_labels)
#print(validation_features.shape)
#print(train_features.shape)

### Create model and train network

In [18]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim= sfvector[0] * sfvector[1] * sfvector[2]))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
Total params: 6,423,812
Trainable params: 6,423,812
Non-trainable params: 0
_________________________________________________________________


In [36]:
optimizer
print(optim)

SGD(lr=0.01, momentum=0.9, decay=5e-4, nesterov=False)


In [ ]:
# SOME INFORMATION ABOUT THE PARAMETERS OF THE ORIGINAL PAPERS:
#DenseNet- keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
#Resnet - keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=False)
#Inception V3 - keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=4e-5, nesterov=False)
#Xception - keras.optimizers.SGD(lr=0.045, momentum=0.9, decay=1e-5, nesterov=False)
#VGG - keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=5e-4, nesterov=False)
##########################################################################################

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=5e-4, nesterov=False)

optim = optim

model.compile(optimizer=optimizer, #optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
 
from keras.callbacks import CSVLogger                           
csv_logger = CSVLogger('TL_history_{}_{}.csv'.format(model_name, datetime.date.today().strftime("%B %d, %Y")),
                       append=True, separator=';')     

tic = time.process_time()

history = model.fit(train_features,
                    train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_features,validation_labels),
                    callbacks=[csv_logger])                   

toc = time.process_time()

print("---- Total Computation time = " + str(datetime.timedelta(seconds=math.ceil(toc-tic))) + " seconds" )

In [40]:
## model to png
# from keras.utils import plot_model
# plot_model(model, to_file='model_{}_{}.png'.format(models_names[0], datetime.date.today().strftime("%B %d, %Y")))

#### Save model weights

In [22]:
# Save model weights
# model.save_weights('bottleneck_fc_model_{}.h5'.format(models_names[0]))

### History and plots

In [23]:
print(history.history.keys())

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


#### Plot

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(range(epochs),history.history['acc'],label='training accuracy', color = "blue")
plt.plot(range(epochs),history.history['val_acc'],label='validation accuracy', color = "red")
plt.legend(loc=0)
plt.xlabel('epochs')
plt.xlim([0,epochs])
plt.ylabel('accuracies on dataset')
plt.grid(True)
plt.title("Training and validation accuracy")
plt.text(epochs*.6, max(history.history['acc'])/(1.25),'''No Transfer Learning, \n 
            model = {} \n
            day = {}, \n
            optimizer = {}, \n
            dataset = {} train, {} val, ({} test), \n
            input shape = {}, \n
            batch size = {}, \n
            epochs = {}, \n
            training_time = {}'''.format(model_name,
                                  datetime.date.today().strftime("%B %d, %Y"),
                                  optim,
                                  nTrain, nVal, nTest,
                                  target_size, #input_shape,
                                  batch_size,
                                  epochs,
                                  str(datetime.timedelta(seconds=math.ceil(toc-tic)))))
plt.show()
fig.savefig('TL_accuracy_{}_{}_{}_imgs.jpg'.format(model_name,
                                                datetime.date.today().strftime("%B %d, %Y"),
                                                nTrain+nVal+nTest))
plt.close(fig)

fig = plt.figure(figsize=(12,8))
plt.plot(range(epochs),history.history['loss'],label='training loss')
plt.plot(range(epochs),history.history['val_loss'],label='validation loss')
plt.legend(loc=0)
plt.xlabel('epochs')
plt.xlim([0,epochs])
plt.ylabel('losses on dataset')
plt.grid(True)
plt.title("Training and validation loss")
plt.text(epochs*.6, max(history.history['loss'])/(2),'''No Transfer Learning, \n 
            model = {} \n
            day = {}, \n
            optimizer = {}, \n
            dataset = {} train, {} val, ({} test), \n
            input shape = {}, \n
            batch size = {}, \n
            epochs = {}, \n
            training_time = {}'''.format(model_name,
                                  datetime.date.today().strftime("%B %d, %Y"),
                                  optim,
                                  nTrain, nVal, nTest,
                                  target_size, #input_shape,
                                  batch_size,
                                  epochs,
                                  str(datetime.timedelta(seconds=math.ceil(toc-tic)))))

plt.show()
fig.savefig('TL_loss_{}_{}_{}_imgs.jpg'.format(model_name,
                                                datetime.date.today().strftime("%B %d, %Y"),
                                                nTrain+nVal+nTest))
plt.close(fig)

#### Test

In [26]:
# TEST AND EVALUATE THE MODEL

datagen = ImageDataGenerator(rescale=1. / 255)
    #featurewise_center=True,
    #featurewise_std_normalization=True)

test_features = np.zeros(shape=(nTest, sfvector[0], sfvector[1], sfvector[2]))
test_labels = np.zeros(shape=(nTest,4))

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(target_size[0], target_size[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

i = 0
for inputs_batch, labels_batch in test_generator:
    features_batch = model_choice.predict(inputs_batch)
    test_features[i * batch_size : (i + 1) * batch_size] = features_batch
    test_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTest:
        break

test_features = np.reshape(test_features, (nTest,reshape_feature_vector))

Found 440 images belonging to 4 classes.


#### Accuracy

In [27]:
print('Predicting on test data')
prediction = np.rint(model.predict(test_features))

print(accuracy_score(test_labels, prediction))

Predicting on test data
0.25


In [28]:
predictions = model.predict_classes(validation_features)
prob = model.predict(validation_features)

### See wich images were predicted wrongly

In [30]:
#EVALUATE FIRST IMAGE OF TEST DATASET
score = model.evaluate(test_features, test_labels, verbose=0)

print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

print('Test Loss', score[0])
print('Test accuracy:', score[1])

test_image = test_features[0:1]
#print(test_image.shape)
print(model.predict(test_image))
print(test_labels[0:1])

acc: 25.00%
Test Loss 12.088571565801448
Test accuracy: 0.25
[[0. 1. 0. 0.]]
[[1. 0. 0. 0.]]


In [31]:
## PREDICT FOR TEST DATASET

Y_pred = model.predict(test_features)
print('\n PROBABILITIES')
print(Y_pred)

y_pred = np.argmax(Y_pred, axis=1)

print('\n RESULTS')
print(y_pred)
  
# y_pred = model.predict_classes(test_features) ## == np-argmax
#print(y_pred)

p=model.predict_proba(test_features) # to predict probability

target_names = ['class 0(EOSINOPHIL)', 'class 1(LYPHOCYTE)', 'class 2(MONOCYTE)', 'class 3(NEUTROPHIL)'] ##REVIEW
print('\n PRECISION, RECALL, SCORE')
print(classification_report(np.argmax(test_labels,axis=1), y_pred,target_names=target_names))

print('\n CONFUSION MATRIX')
print(confusion_matrix(np.argmax(test_labels,axis=1), y_pred))
#print(confusion_matrix(np.argmax(test_labels,axis=1), y_pred['EOSINOPHIL', 'MONOCYTE','LIMPHOCYTE', 'NEUTROPHIL']))


 PROBABILITIES
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]

 RESULTS
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

/home/deep/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
print(str(accuracy_score(test_labels, prediction)))

0.25


In [33]:
file2write=open('Test results_{}_#testexamples:{}_{}'.format(model_name,
                                                                  len(Y_pred),
                                                                  datetime.date.today().strftime("%B %d, %Y")),'w')
file2write.write('Predicting on test data' + str(accuracy_score(test_labels, prediction)))
file2write.write('EVALUATE FIRST IMAGE OF TEST DATASET')
file2write.write("%s: %.2f%%" %(model.metrics_names[1], score[1]*100))
file2write.write('Test Loss:{}'.format(score[0]))
file2write.write('Test accuracy:{}'.format(score[1]))
file2write.write(str(model.predict(test_image)))
file2write.write(str(test_labels[0:1]))
file2write.write('PREDICT FOR TEST DATASET')
file2write.write('\n PROBABILITIES \n {}'.format(Y_pred))
file2write.write('\n RESULTS \n {}'.format(y_pred))
file2write.write('\n PRECISION, RECALL, SCORE \n' + classification_report(np.argmax(test_labels,axis=1), y_pred,target_names=target_names))
file2write.write('\n CONFUSION MATRIX \n {}'.format(confusion_matrix(np.argmax(test_labels,axis=1), y_pred)))
file2write.close()

/home/deep/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
 keras.backend.clear_session()